In [1]:
# https://www.youtube.com/watch?v=GIsg-ZUy0MY
import torch
import numpy as np

In [2]:
torch.from_numpy

# output = x * w1 + y * w2 + b1
# weights adjust for variables, bias adjusts for units

<function _VariableFunctions.from_numpy>

In [3]:
# Input input data (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')


In [4]:
# Targets yield of (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
# Convert to pytorch tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [6]:
# The (2, 3) because there are 3 input datas (temp rainfall and humidity) on two outputs (apples oranges) so it is a 2x3 matrix.

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.1865, -1.1905,  1.0061],
        [ 1.4312,  1.8770,  0.7243]], requires_grad=True)
tensor([ 1.0973, -1.5434], requires_grad=True)


In [7]:
# The model does a matrix multiplication between the input variables and the weights plus the bias.
# The @ does matrix multiplication in pytorch


def model(x):
    return x @ w.t() + b

In [8]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[-122.0222,  259.8371],
        [-147.2522,  340.2256],
        [-203.3078,  416.4949],
        [-133.8958,  251.9498],
        [-124.6358,  328.1002]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [9]:
# Need to improve the weights and bias
# Loss function evaluates the accuracy of the model.
# take the difference, square all elements to remove negatives, and calculate the average
# Mean squared error loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()
 # .numel gives number of elements



In [10]:
loss = mse(preds, targets)
print(loss)

tensor(52735.7500, grad_fn=<DivBackward0>)


In [11]:
loss.backward()

In [12]:
# w.grad are the gradients for the weights (derivative of the loss)
print(w)
print(w.grad)
# if gradient is positive decrease weight (to find inverse peak)
# if gradient is negative increase weight (to find inverse peak)

tensor([[-1.1865, -1.1905,  1.0061],
        [ 1.4312,  1.8770,  0.7243]], requires_grad=True)
tensor([[-18683.1191, -20751.8984, -12531.9199],
        [ 19328.8945,  20215.1406,  12501.2578]])


In [13]:
print(b)
print(b.grad)

tensor([ 1.0973, -1.5434], requires_grad=True)
tensor([-222.4228,  227.3215])


In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
# Adjust weights and biases using gradeint decent:
# 1 Gen predictions
# 2 Calc loss
# 3 Compute gradients w.r.t the w and b
# 4 Adjust weights by subtraction
# 5 Reset gradients 



In [16]:
preds = model(inputs)
print(preds)

tensor([[-122.0222,  259.8371],
        [-147.2522,  340.2256],
        [-203.3078,  416.4949],
        [-133.8958,  251.9498],
        [-124.6358,  328.1002]], grad_fn=<AddBackward0>)


In [17]:
loss = mse(preds, targets)
print(loss)

tensor(52735.7500, grad_fn=<DivBackward0>)


In [18]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-18683.1191, -20751.8984, -12531.9199],
        [ 19328.8945,  20215.1406,  12501.2578]])
tensor([-222.4228,  227.3215])


In [19]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
    

In [20]:
print(w)
print(b)

tensor([[-0.9997, -0.9830,  1.1315],
        [ 1.2379,  1.6748,  0.5993]], requires_grad=True)
tensor([ 1.0995, -1.5457], requires_grad=True)


In [21]:
# train for 100 epochs. Hyperparameters are 100 epochs and the 1e-5 value
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()


In [22]:
# see if loss has improved
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(167.5224, grad_fn=<DivBackward0>)


In [23]:
preds

tensor([[ 59.4577,  73.2480],
        [ 91.8240,  96.9299],
        [ 93.1386, 136.7352],
        [ 34.1424,  55.0342],
        [111.2476, 101.8088]], grad_fn=<AddBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [25]:
# Linear regression using PyTorch buit-ins 

In [26]:
import torch.nn as nn

In [27]:
# new data
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [28]:
from torch.utils.data import TensorDataset

In [29]:
# access rows as tuples or pairs to split up for train/test
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [30]:
from torch.utils.data import DataLoader

In [31]:
# Define the loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# shuffle randomizes the order of the data

In [32]:
for xb, yb in train_dl:
    print(xb)
    print(yb)

tensor([[ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [103., 119.],
        [ 56.,  70.],
        [103., 119.],
        [ 56.,  70.]])
tensor([[102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 56.,  70.],
        [ 22.,  37.],
        [119., 133.],
        [103., 119.]])
tensor([[ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]])
tensor([[ 81., 101.],
        [119., 133.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 81., 101.]])


In [33]:
# Define model
model = nn.Linear(3, 2) # specify inputs (temp, humidity, water) and outputs (apples, oranges)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2420, -0.1077, -0.2508],
        [-0.4374, -0.4702, -0.0941]], requires_grad=True)
Parameter containing:
tensor([0.3938, 0.1354], requires_grad=True)


In [34]:
# all weights and biases
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2420, -0.1077, -0.2508],
         [-0.4374, -0.4702, -0.0941]], requires_grad=True),
 Parameter containing:
 tensor([0.3938, 0.1354], requires_grad=True)]

In [35]:
preds = model(inputs)
preds

tensor([[ 6.0665e-02, -6.7345e+01],
        [-3.1119e+00, -8.7069e+01],
        [-7.5293e+00, -1.0639e+02],
        [ 1.1170e+01, -6.8178e+01],
        [-1.0803e+01, -8.1774e+01],
        [ 6.0665e-02, -6.7345e+01],
        [-3.1119e+00, -8.7069e+01],
        [-7.5293e+00, -1.0639e+02],
        [ 1.1170e+01, -6.8178e+01],
        [-1.0803e+01, -8.1774e+01],
        [ 6.0665e-02, -6.7345e+01],
        [-3.1119e+00, -8.7069e+01],
        [-7.5293e+00, -1.0639e+02],
        [ 1.1170e+01, -6.8178e+01],
        [-1.0803e+01, -8.1774e+01]], grad_fn=<AddmmBackward>)

In [36]:
# Loss function
import torch.nn.functional as F

In [37]:
loss_fn = F.mse_loss

In [38]:
# Documentation
?nn.Linear

In [39]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(20219.4453, grad_fn=<MseLossBackward>)


In [42]:
# Optimizer 
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [48]:
# Everything
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # gen predictions
            pred = model(xb)
            # calc loss
            loss = loss_fn(pred, yb)
            # compute gradients
            loss.backward()
            # update parameters using gradients
            opt.step()
            # reset grads to 0
            opt.zero_grad()
            
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [49]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 171.5510
Epoch [20/100], Loss: 72.8533
Epoch [30/100], Loss: 47.5551
Epoch [40/100], Loss: 54.1487
Epoch [50/100], Loss: 101.3101
Epoch [60/100], Loss: 57.8774
Epoch [70/100], Loss: 21.8656
Epoch [80/100], Loss: 30.2975
Epoch [90/100], Loss: 12.3881
Epoch [100/100], Loss: 33.3720
